In [1]:
using FermiCG
using Printf
using Test
using JLD2 

@load "../test/_testdata_cmf_h12_64bit.jld2"

clustered_ham = FermiCG.extract_ClusteredTerms(ints, clusters)
cluster_ops = FermiCG.compute_cluster_ops(cluster_bases, ints);
FermiCG.add_cmf_operators!(cluster_ops, cluster_bases, ints, d1.a, d1.b);

v_bst = FermiCG.BSTstate(clusters, FockConfig(init_fspace), cluster_bases)
v_tps = FermiCG.TPSCIstate(clusters, FockConfig(init_fspace))
FermiCG.eye!(v_tps)

s_bst = FermiCG.build_compressed_1st_order_state(v_bst, cluster_ops, clustered_ham, nbody=4, thresh=1e-24)
s_tps = FermiCG.open_matvec_thread(v_tps, cluster_ops, clustered_ham, thresh=1e-32)

@show size(s_bst);
@show size(s_tps);





 Remove duplicates


 Number of terms reduced from  2525 to  1105
IDX001:DIM0016:

001|002|


IDX002:DIM0016:003|004|


IDX003:DIM0256:005|006|007|008|


...

 Compute matrix-vector:                            

  3.284795 seconds (15.26 M allocations: 733.249 MiB, 3.94% gc time, 99.80% compilation time)


 Now collect thread results :                      

  0.011717 seconds (1.96 k allocations: 113.500 KiB, 98.03% compilation time)


size(s_bst) = (16298, 1)
size(s_tps) = (49378, 1)


# BST PT2

In [2]:
e_var = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);
e_pt = FermiCG.compute_pt2_energy(v_bst, cluster_ops, clustered_ham, thresh_foi=1e-6, max_iter=100, tol=1e-12, verbose=0);

 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                

  1.574623 seconds (3.06 M allocations: 183.109 MiB, 2.06% gc time, 99.25% compilation time)


 Iter:   1 SS: 1    E: 

 -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 


 Diagonalization time:                               0.617404 seconds
 Compute <S^2>:                                    

...

 Compute <0|H0|0>:                                 

  0.008108 seconds (8.92 k allocations: 843.528 KiB, 91.27% compilation time)


 Number of jobs:    525
 Number of threads: 4


----------------------------------------------------------------------------------------------------

 Time spent computing E2:                               5.9 s Allocated:    2.0e+00 GB
 State   1: E(PT2) corr:                          -0.00988248
 State   1: E(PT2):                              -18.32699143
 ==================================================================|


In [54]:
e_var, v = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);

s_bst = FermiCG.build_compressed_1st_order_state(v, cluster_ops, clustered_ham, nbody=4, thresh=1e-33)
    
# FermiCG.project_out!(s_bst, v, thresh=1e-32)
# @show size(s_bst)
# s_bst = FermiCG.compress(s_bst, thresh=1e-32)
# @show size(s_bst)
v_pt, e_pt = FermiCG.hylleraas_compressed_mp2(s_bst, v, cluster_ops, clustered_ham, tol=1e-8, verbose=1);

 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                

  0.261886 seconds (4.08 k allocations: 30.747 MiB, 95.69% gc time)


 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 


 Diagonalization time:                               0.001251 seconds
 Compute <S^2>:                                    

  0.000508 seconds (1.35 k allocations: 399.688 KiB)


...

 11	1.79e-07


 12	6.35e-08


 13	2.16e-08


 14	5.91e-09

 Time to solve for PT1 with conjugate gradient:      0.104574 seconds   8.94e-02 Gb


 Compute <0|H|1>:                                    0.032074 seconds   8.11e-03 Gb
 SxC[r]   0.00000000 SxSx   1.00000000
 State   1: E(PT2) corr:                          -0.00989530
 State   1: E(PT2):                              -18.32700425


In [ ]:
e_pt, v_pt = FermiCG.do_fois_pt2(v_bst, cluster_ops, clustered_ham, thresh_foi=1e-3, max_iter=50, tol=1e-9);

# Pseudo-Canonicalization

In [61]:
e_var, v = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);

s_bst = FermiCG.build_compressed_1st_order_state(v, cluster_ops, clustered_ham, nbody=4, thresh=1e-33)
    
FermiCG.pseudo_canon_pt1(s_bst, v, cluster_ops, clustered_ham, verbose=1);
# v_pt, e_pt = FermiCG.pseudo_canon_pt1(s_bst, v, cluster_ops, clustered_ham, tol=1e-8, verbose=1);


 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                

  0.002403 seconds (4.08 k allocations: 30.747 MiB)


 Iter:   1 SS: 1    E:  -18.31710895*  R: 

0.0e+00*  LinDep: 0.0e+00* 


 Diagonalization time:                               0.001657 seconds
 Compute <S^2>:                                    

...

  Root       Energy           S2
     1 -18.31710895   0.00000000
 ==================================================================|
 Number of tasks:                                         661
 Compute tasks:                                    
 Time building compressed vector:                    0.042541 seconds   6.24e-02 Gb GC: 0.0e+00 sec


 Add results together:                               0.038283 seconds   1.76e-02 Gb GC: 0.0e+00 sec


 Length of input      FOIS:                             16396


 Compute <X|H|0>:                                    0.013091 seconds   1.10e-02 Gb
  Root      <0|H|0>      <0|F|0>
     1 -18.31710895 -10.81361409
 Cache zeroth-order Hamiltonian:                     0.013091 seconds   1.10e-02 Gb
1441.9153951042406


 Compute <0|H|1>:                                    0.028152 seconds   1.62e-02 Gb
 SxC[r]   0.00000000 SxSx   1.00000000
 State   1: E(PT2) corr:                          -0.00992618
 State   1: E(PT2):                              -18.32703513
